In [24]:
import pandas as pd

dataset = pd.read_csv('data/cleaned-data.csv', index_col=0).dropna()

In [25]:
%matplotlib inline

import json
import math
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas

from sklearn.feature_extraction.text import CountVectorizer

#для сопоставления словам их tfIdf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn_deltatfidf import DeltaTfidfVectorizer
from sklearn.metrics import classification_report

In [26]:
# n-граммная схема, к примеру (1, 3) значит - униграммы + биграммы + триграммы
ngram_scheme = (1, 2)

In [27]:
#для сопоставления словам их количества в твите
from sklearn.feature_extraction.text import CountVectorizer

#для сопоставления словам их tfIdf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn_deltatfidf import DeltaTfidfVectorizer

In [28]:
import warnings
warnings.filterwarnings("ignore")


In [29]:
dataset['text'][20001:20020]
dataset['label'][20001:20020]

20001    1
20002    1
20003    1
20004    1
20005    1
20006    1
20007    1
20008    1
20009    1
20010    1
20011    1
20012    1
20013    1
20014    1
20015    1
20016    1
20017    1
20018    1
20019    1
Name: label, dtype: int64

In [30]:
print('N-gram Scheme:', ngram_scheme)

count_vectorizer = CountVectorizer(analyzer = "word", ngram_range=ngram_scheme) 
#delta_vectorizer = DeltaTfidfVectorizer(analyzer='word', ngram_range=ngram_scheme)
tf_vectorizer = TfidfVectorizer(analyzer='word', ngram_range=ngram_scheme)

N-gram Scheme: (1, 2)


In [31]:
vectorizer = tf_vectorizer
#vectorizer = delta_vectorizer
y = dataset['label']
X = vectorizer.fit_transform(dataset['text'])
dataset['text'][20000]

20000    сегодня был день супер море эмоций вечер конеч...
20000    очень обидно когда подруга начинает репостить ...
Name: text, dtype: object

In [32]:
#Запускаем классификатор
X_train, X_test, y_train, y_test =\
 train_test_split(X, dataset['label'], test_size=0.3)

#cv = ShuffleSplit(n_splits=10, test_size=0.3, random_state=0)

start_time = time.time()
from sklearn import linear_model

In [33]:

clf = linear_model.SGDClassifier(alpha=1e-05, loss='hinge', penalty='l2')
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
print(pred[1])

1


In [34]:
%%time
clf = linear_model.SGDClassifier(alpha=1e-05, loss='hinge', penalty='l2')
clf.fit(X_train, y_train)
pred = clf.predict(X_test)


from sklearn.metrics import classification_report
print(classification_report(y_test, pred))
print (clf.score(X_test, y_test))



import pickle as cPickle

with open('my_classifier.pkl', 'wb') as fid:
    cPickle.dump(clf, fid)    
with open('my_vectorizer.pkl', 'wb') as vect:
    cPickle.dump(vectorizer, vect)  

# load it again
with open('my_classifier.pkl', 'rb') as fid:
    clf_loaded = cPickle.load(fid)

pred1 = clf_loaded.predict(X_test)
print(classification_report(y_test, pred1))
print (clf_loaded.score(X_test, y_test))


feature_to_coef = {
    word: coef for word, coef in zip(
        vectorizer.get_feature_names(), clf.coef_[0]
    )
}
for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:10]:
    print (best_positive)
    
#     ('excellent', 0.9288812418118644)
#     ('perfect', 0.7934641227980576)
#     ('great', 0.675040909917553)
#     ('amazing', 0.6160398142631545)
#     ('superb', 0.6063967799425831)
    
for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1])[:10]:
    print (best_negative)

texts = [u'у меня всё хорошо',
         u'что делаешь сейчас',
         u'как же всё плохо ахаха']
p = vectorizer.transform(texts)
pred2 = clf.predict(p)
print(pred2)

              precision    recall  f1-score   support

          -1       0.78      0.71      0.74     33347
           1       0.74      0.81      0.77     34669

   micro avg       0.76      0.76      0.76     68016
   macro avg       0.76      0.76      0.76     68016
weighted avg       0.76      0.76      0.76     68016

0.758277464126088
              precision    recall  f1-score   support

          -1       0.78      0.71      0.74     33347
           1       0.74      0.81      0.77     34669

   micro avg       0.76      0.76      0.76     68016
   macro avg       0.76      0.76      0.76     68016
weighted avg       0.76      0.76      0.76     68016

0.758277464126088
('dd', 6.759496681076041)
('ddd', 5.674772256172816)
('ахахах', 4.226849764665232)
('ахах', 4.134681624695558)
('ахаха', 3.9924989695506086)
('не плохо', 3.888750232306505)
('не зря', 3.8171341397915843)
('приятно', 3.7900635317035722)
('не за', 3.7262001964775697)
('обожаю', 3.617948494001131)
('oo', -8.1744

In [59]:
X_test[:1].toarray()


array([[0., 0., 0., ..., 0., 0., 0.]])

NameError: name 'clf' is not defined

In [201]:
print (clf.score(X_train, y_train))

0.8730246707522155


In [84]:
#наивный байес
clf = MultinomialNB()
print('10-fold cross validation:\n')
start_time = time.time()

scores = cross_val_score(clf, X, y, cv=cv.split(X), scoring ='f1')
print(" f1: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), 'MultinomialNB'))

recall_score = cross_val_score(clf, X,y, cv=cv.split(X), scoring ='recall')
print(" recall: %0.2f (+/- %0.2f) [%s]" % (recall_score.mean(), recall_score.std(), 'MultinomialNB'))

precision_score = cross_val_score(clf, X,y, cv=cv.split(X), scoring='precision')
print(" precision: %0.2f (+/- %0.2f) [%s]" % (precision_score.mean(), precision_score.std(), 'MultinomialNB'))

NB_res = cross_val_score(clf, X, y, cv=10)
print(" Accuracy [%s]" %(NB_res.mean()))
print("---Classifier %s use %s seconds ---" %('MultinomialNB', (time.time() - start_time)))

10-fold cross validation:

 f1: 0.73 (+/- 0.00) [MultinomialNB]
 recall: 0.71 (+/- 0.00) [MultinomialNB]
 precision: 0.75 (+/- 0.00) [MultinomialNB]
 Accuracy [0.7274409929371912]
---Classifier MultinomialNB use 13.369346857070923 seconds ---
